In [37]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics


import nltk
nltk.download('stopwords')

import string
from nltk.corpus import stopwords



from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
df=pd.read_csv('spam.csv', encoding='latin-1')
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [39]:
df.dropna(how="any", inplace=True, axis=1)
df.columns = ['label', 'message']

In [40]:
df['label_num'] = df.label.map({'ham':0, 'spam':1})
df.head(g)

,label,message,label_num
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [41]:
def text_process(mess):
    """
    Takes in a string of text and performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    STOPWORDS = set(stopwords.words('english'))  # Make sure to use set() to improve performance
    # Remove punctuation
    nopunc = [char for char in mess if char not in string.punctuation]
    # Join the characters again to form the string
    nopunc = ''.join(nopunc)
    # Remove stopwords and return as lowercase
    return ' '.join([word.lower() for word in nopunc.split() if word.lower() not in STOPWORDS])


In [42]:
df['clean_msg'] = df['message'].apply(text_process)


In [43]:
X = df.clean_msg
y = df.label_num

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [44]:
# Define the pipeline with Logistic Regression model
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LogisticRegression(solver='liblinear'))])

In [45]:
# Train the model
pipe.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('model', LogisticRegression(solver='liblinear'))])

In [46]:
# Make predictions on the test set
y_pred = pipe.predict(X_test)

In [47]:
# Evaluate the model
print("Accuracy Score:", metrics.accuracy_score(y_test, y_pred))
print("Confusion Matrix:")
print(metrics.confusion_matrix(y_test, y_pred))
print("Classification Report:")
print(metrics.classification_report(y_test, y_pred))

Accuracy Score: 0.9619526202440776
Confusion Matrix:
[[1208    5]
 [  48  132]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1213
           1       0.96      0.73      0.83       180

    accuracy                           0.96      1393
   macro avg       0.96      0.86      0.91      1393
weighted avg       0.96      0.96      0.96      1393

